# 1. Data Preparation

In [55]:
import pandas as pd
import os
debug = True


####  Load Data from files

In [56]:
#Code to Load Data in a DataFrame
datafolder = '../data/'
#print(os.listdir(datafolder))
train_features_file_name = 'train_features.csv'
train_labels_file_name = 'train_salaries.csv'
test_features_file_name = 'test_features.csv'

# with open(os.path.join(datafolder, test_features_file_name)) as f:
#     x = f.readlines()
# print(x[1])
train_data_features = pd.read_csv(os.path.join(datafolder, train_features_file_name))
train_data_labels = pd.read_csv(os.path.join(datafolder, train_labels_file_name))
train_full = pd.merge(train_data_features, train_data_labels, on='jobId',how ='inner')
test_data_features = pd.read_csv(os.path.join(datafolder, test_features_file_name))

assert(train_full.shape[0] == train_data_features.shape[0])
assert(train_full.shape[0] == train_data_labels.shape[0])
#train_full.dropna(how ='any', inplace = True)
train_full.shape

(1000000, 9)

####  Encode Categorical Variables

One Hot Encoding

In [57]:
#Encode Categorical Variables One Hot

def one_hot_encoding(c, df):
    new_f_name = 'OH_'+ c
    df = pd.concat([train_full, pd.get_dummies(df[c], prefix= new_f_name)], axis=1)
    return df, new_f_name

cat_features_normal = ['companyId', 'major', 'industry']
cat_features_normal_new = []
for c in cat_features_normal:
    train_full, new_f_name = one_hot_encoding(c, train_full)
    cat_features_normal_new.append(new_f_name)
    
    
print(cat_features_normal_new)
#train_full.head(1)

['OH_companyId', 'OH_major', 'OH_industry']


Ordinal  Encoding

In [58]:
#Pending Visualize data to decide order

In [59]:
#Encode Categorical Variables Ordered

def ordinal_encoding(c, df, lookup):
    new_f_name = 'ORD_'+ c
    df[new_f_name] =  df[c].map(lookup)
    return df, new_f_name

degreeOrder = {'BACHELORS':2, 'DOCTORAL':4, 'HIGH_SCHOOL':1, 'MASTERS':3, 'NONE':0}
jobTypeOrder = {'CEO':7, 'CFO':6, 'CTO':5, 'VICE_PRESIDENT':4, 'MANAGER':3,'SENIOR':2, 'JUNIOR':1, 'JANITOR':0}

cat_features_ordinal = ['jobType', 'degree']
cat_features_ordinal_new = []
for c in cat_features_ordinal:
    train_full, new_f_name = ordinal_encoding(c, train_full, eval(c+'Order'))
    cat_features_ordinal_new.append(new_f_name)

print(cat_features_ordinal_new)
#train_full.head(1)

['ORD_jobType', 'ORD_degree']


In [75]:
#Shuffle Train Data
train_full =  train_full.sample(frac=1).reset_index(drop=True)

In [60]:
# Split Data into K Folds
from sklearn.model_selection import KFold
def GetKFoldData(df,k):
    folds = {}
    kfolds = KFold(n_splits=k, shuffle = True, random_state = 4) 
    foldidx = 0
    for train_idx, test_idx in kfolds.split(df.index):
        folds[foldidx] = { 'train': df.iloc[train_idx], 'test':df.iloc[test_idx]}
        foldidx+=1
    return folds

#exp_data_k_folds = GetKFoldData(train_full, 10)
#idx_list_org = exp_data_k_folds[0]['test'].index.tolist()
#exp_data_k_folds[0]['test'].shape


In [61]:
# set(test_data_features.jobType.tolist())
# set(test_data_features.degree.tolist())

# 2. Create ML Models

In [64]:
# Select Features
train_features = ['yearsExperience', 'milesFromMetropolis']
for f in train_full.columns:
    if 'OH' in f or 'ORD' in f:
        train_features.append(f)
        
#train_full[train_features].head()

,yearsExperience,milesFromMetropolis,OH_companyId_COMP0,OH_companyId_COMP1,OH_companyId_COMP10,OH_companyId_COMP11,OH_companyId_COMP12,OH_companyId_COMP13,OH_companyId_COMP14,OH_companyId_COMP15,...,OH_major_PHYSICS,OH_industry_AUTO,OH_industry_EDUCATION,OH_industry_FINANCE,OH_industry_HEALTH,OH_industry_OIL,OH_industry_SERVICE,OH_industry_WEB,ORD_jobType,ORD_degree
0,10,83,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,6,3
1,3,73,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,7,1
2,10,38,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,4,4
3,8,17,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,3,4
4,8,16,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,4,2


In [73]:
# Different Regression Models Evaluated
from sklearn.tree import DecisionTreeRegressor
models = {
           'DT' : {'modelobj' :  DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)}
         }


In [74]:
# Functions to Evaluate a Model
from sklearn.metrics import mean_squared_error as MSE
def EvalModel(model, curr_fold, train_features):
    curr_fold_X_train = curr_fold['train'][train_features]
    curr_fold_y_train = curr_fold['train']['salary']

    curr_fold_X_test = curr_fold['test'][train_features]
    curr_fold_y_test = curr_fold['test']['salary']
    
    curr_model = model['modelobj'].fit(curr_fold_X_train, curr_fold_y_train)
    
    curr_test_predict = curr_model.predict(curr_fold_X_test)
    curr_fold_mse = MSE(curr_test_predict, curr_fold_y_test)
    curr_fold_rmse = curr_fold_mse**0.5
    print("Test set RMSE of dt: {:.2f}".format(curr_fold_rmse))
    
    

In [72]:
# Run Experiments Over Various Models
def RunExp(k, models ,train_features):
    kfolddata =  GetKFoldData(train_full, k)
    for curr_fold in kfolddata:
        for m in models:
            EvalModel(models[m], kfolddata[curr_fold], train_features)


            
            
RunExp(10, models, train_features)


Test set RMSE of dt: 29.11
Test set RMSE of dt: 28.98
Test set RMSE of dt: 29.14
Test set RMSE of dt: 29.13
Test set RMSE of dt: 29.02
Test set RMSE of dt: 29.20
Test set RMSE of dt: 29.16
Test set RMSE of dt: 29.08
Test set RMSE of dt: 29.27
Test set RMSE of dt: 29.05
